In [1]:
from regime_module import *
from statsmodels.iolib.summary2 import summary_col
import csv
import os

cur_wd = os.getcwd()
print(cur_wd)
outname = 'output_time_series.txt'

D:\janko_machine_learning\playground


In [2]:
ff_factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', index_col = 0)
ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m')
ff_factors = ff_factors / 100.0
ff_factors[ff_factors == -999] = np.NaN
ff_factors[ff_factors == -99.9] = np.NaN
ff_factors = ff_factors.dropna(how = 'any', axis = 0)
ff_factors = ff_factors[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
ff_factors_compare = ff_factors.copy()

# hoe zue zhang

In [3]:
q_factor = pd.read_csv('q5_factors.csv', index_col = 0)
q_factor.index = pd.to_datetime(q_factor.index, format= '%Y%m')
q_factor = q_factor / 100.0

factor_model = q_factor.copy()
factor_model = factor_model[['R_MKT', 'R_ME', 'R_IA', 'R_ROE', 'R_EG']]
rf = q_factor[['R_F']]

index_symbol = 'R_MKT'

print(q_factor.head(5))

                 R_F     R_MKT      R_ME      R_IA     R_ROE      R_EG
1967-01-01  0.003927  0.081603  0.067282 -0.029434  0.018866 -0.022469
1967-02-01  0.003743  0.007202  0.016918 -0.002064  0.035317  0.025812
1967-03-01  0.003693  0.039691  0.019517 -0.016933  0.018876 -0.014789
1967-04-01  0.003344  0.038265 -0.007446 -0.029519  0.010983 -0.022351
1967-05-01  0.003126 -0.042419  0.029132  0.024686  0.005234  0.002749


In [4]:
#start date for data
start = '2004-01-01'
#end date for data
end = '2020-01-01'

#the lookback for alpha feature
lookback = 60
#control the leverage
lev = 1.
stk_symbols  = factor_model.columns
returns = factor_model.copy()


#set the backtest parameters
bp = BacktestParameters(stk_symbols, start, end, lookback, lev  )
bt = Backtest(bp, returns, mode = 'insample')
data = Data(bp)

df_alpha, idiosyncratic_vol, binary_alpha = data.get_alpha(returns, factor_model, rf)
rolling_mean_12, rolling_mean_6, rolling_std_12, binary_mean_12,binary_mean_6,binary_returns = data.get_rolling_features(returns, 0.0)


#creates a dictionary of different features for analysis based on ticker keys
feature_list = [binary_alpha]

feature_set = data.get_features(feature_list)


C:\Users\jjanko\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


               R_MKT      R_ME      R_IA     R_ROE      R_EG
1972-02-01 -0.006003 -0.000718 -0.000272  0.006437  0.005032
1972-03-01 -0.005190  0.000011 -0.000285  0.005517  0.005497
1972-04-01 -0.005061 -0.000321 -0.000492  0.005074  0.005028
1972-05-01 -0.005651  0.000034  0.000432  0.005457  0.005369
1972-06-01 -0.003984 -0.000320 -0.000824  0.006204  0.005462
               R_MKT      R_ME      R_IA     R_ROE      R_EG
1972-02-01  0.044118  0.026733  0.020240  0.018881  0.019336
1972-03-01  0.044233  0.027048  0.020240  0.020261  0.019037
1972-04-01  0.044164  0.027120  0.020062  0.020322  0.019038
1972-05-01  0.043933  0.026998  0.019858  0.020589  0.018904
1972-06-01  0.043656  0.026960  0.019462  0.020558  0.018900


In [6]:
output_reg = []
output_capm = []
names = ['KNN L/S', 'KNN L', 'KNN S', 'Neural Net L/S', 'Neural Net L', 'Neural Net S', 'RF L/S', 'RF L', 'RF S', 'Ensemble', 'Binary']

data_type = 'q_factor'

print("KNN")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'ls' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'l' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 's' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Neural Net")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'ls' + '.csv'))

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'l' + '.csv'))

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 's' + '.csv'))

print("Random Forest")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'ls' + '.csv'))

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'l' + '.csv'))

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 's' + '.csv'))

print("Ensemble System")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors", "Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'yes')  
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_ensemble' '_' + 'ls' + '.csv'))
    
print("Simple Binary Decision")
df_alpha_shift = df_alpha.shift(1).dropna(axis = 0)
backtest = returns.loc[df_alpha_shift.index]
weights = returns.loc[df_alpha_shift.index]
for count, row in  backtest.iterrows(): 
    for j in backtest.columns:
        if df_alpha_shift.loc[count, j] >= 0:
            backtest.loc[count, j] = backtest.loc[count, j]
            weights.loc[count, j] = 1.0
        else:
            backtest.loc[count, j] = -backtest.loc[count, j]
            weights.loc[count, j] = -1.0
            
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_simp_binary' '_' + 'ls' + '.csv'))


print(summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names))

KNN
Long and Short
                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.033
Model:                            OLS   Adj. R-squared:                  0.024
Method:                 Least Squares   F-statistic:                     2.147
Date:                Sat, 30 Jan 2021   Prob (F-statistic):             0.0586
Time:                        11:13:23   Log-Likelihood:                 1554.2
No. Observations:                 563   AIC:                            -3096.
Df Residuals:                     557   BIC:                            -3070.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0282      0.030 

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.086
Model:                            OLS   Adj. R-squared:                  0.078
Method:                 Least Squares   F-statistic:                     6.137
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           1.51e-05
Time:                        11:16:26   Log-Likelihood:                 1632.1
No. Observations:                 563   AIC:                            -3252.
Df Residuals:                     557   BIC:                            -3226.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0345      0.028      1.244      0.2

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.072
Model:                            OLS   Adj. R-squared:                  0.064
Method:                 Least Squares   F-statistic:                     4.194
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           0.000947
Time:                        11:22:04   Log-Likelihood:                 1566.7
No. Observations:                 563   AIC:                            -3121.
Df Residuals:                     557   BIC:                            -3095.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0046      0.027      0.168      0.8

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.082
Model:                            OLS   Adj. R-squared:                  0.073
Method:                 Least Squares   F-statistic:                     4.761
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           0.000286
Time:                        11:22:42   Log-Likelihood:                 1614.3
No. Observations:                 563   AIC:                            -3217.
Df Residuals:                     557   BIC:                            -3191.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0673      0.026      2.542      0.0

In [7]:
resultFile = open(outname,'w')
resultFile.close()
q_factor_results = summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names)
q_factor_results.title = 'q factor'
q_factor_results_capm = summary_col(output_capm,stars=True,float_format='%0.4f',model_names=names)
q_factor_results_capm.title = 'q factor capm'
resultFile = open(outname,'a')
resultFile.write(q_factor_results.as_text())
resultFile.write('\n')
resultFile.write(q_factor_results_capm.as_text())
resultFile.write('\n')
resultFile.close()

# Fama French 5 Factor

In [8]:
ff_factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', index_col = 0)
ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m')
ff_factors = ff_factors / 100.0
ff_factors[ff_factors == -999] = np.NaN
ff_factors[ff_factors == -99.9] = np.NaN
ff_factors = ff_factors.dropna(how = 'any', axis = 0)

factor_model = ff_factors.copy()
factor_model = factor_model[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
rf = ff_factors[['RF']]

index_symbol = 'Mkt-RF'

print(ff_factors.head(5))

#start date for data
start = '2004-01-01'
#end date for data
end = '2020-01-01'

#the lookback for alpha feature
lookback = 60
#control the leverage
lev = 1.
stk_symbols  = factor_model.columns
returns = factor_model.copy()


#set the backtest parameters
bp = BacktestParameters(stk_symbols, start, end, lookback, lev  )
bt = Backtest(bp, returns, mode = 'insample')
data = Data(bp)

df_alpha, idiosyncratic_vol, binary_alpha = data.get_alpha(returns, factor_model, rf)
rolling_mean_12, rolling_mean_6, rolling_std_12, binary_mean_12,binary_mean_6,binary_returns = data.get_rolling_features(returns, 0.0)


#creates a dictionary of different features for analysis based on ticker keys
feature_list = [binary_alpha]

feature_set = data.get_features(feature_list)

            Mkt-RF     SMB     HML     RMW     CMA      RF
1963-07-01 -0.0039 -0.0047 -0.0083  0.0066 -0.0115  0.0027
1963-08-01  0.0507 -0.0079  0.0167  0.0040 -0.0040  0.0025
1963-09-01 -0.0157 -0.0048  0.0018 -0.0076  0.0024  0.0027
1963-10-01  0.0253 -0.0129 -0.0010  0.0275 -0.0224  0.0029
1963-11-01 -0.0085 -0.0084  0.0171 -0.0045  0.0222  0.0027


C:\Users\jjanko\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


              Mkt-RF       SMB       HML       RMW       CMA
1968-08-01  0.004686  0.006016 -0.000766 -0.002048 -0.006084
1968-09-01  0.003146  0.006134 -0.000267 -0.002788 -0.005145
1968-10-01  0.003727  0.006555 -0.000347 -0.002577 -0.005342
1968-11-01  0.003766  0.007209 -0.000303 -0.003457 -0.004763
1968-12-01  0.004546  0.007290 -0.000566 -0.003387 -0.004722
              Mkt-RF       SMB       HML       RMW       CMA
1968-08-01  0.029955  0.027251  0.017452  0.014173  0.019689
1968-09-01  0.029701  0.027173  0.018432  0.014811  0.020400
1968-10-01  0.029452  0.027096  0.018422  0.014703  0.020372
1968-11-01  0.029710  0.027022  0.018420  0.014432  0.020215
1968-12-01  0.029435  0.026727  0.018434  0.014492  0.020312


In [9]:
output_reg = []
output_capm = []
names = ['KNN L/S', 'KNN L', 'KNN S', 'Neural Net L/S', 'Neural Net L', 'Neural Net S', 'RF L/S', 'RF L', 'RF S', 'Ensemble', 'Binary']

data_type = 'ff5'

print("KNN")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'ls' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'l' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 's' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Neural Net")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'ls' + '.csv'))

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'l' + '.csv'))

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 's' + '.csv'))

print("Random Forest")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'ls' + '.csv'))

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'l' + '.csv'))

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 's' + '.csv'))

print("Ensemble System")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors", "Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'yes')  
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_ensemble' '_' + 'ls' + '.csv'))
    
print("Simple Binary Decision")
df_alpha_shift = df_alpha.shift(1).dropna(axis = 0)
backtest = returns.loc[df_alpha_shift.index]
weights = returns.loc[df_alpha_shift.index]
for count, row in  backtest.iterrows(): 
    for j in backtest.columns:
        if df_alpha_shift.loc[count, j] >= 0:
            backtest.loc[count, j] = backtest.loc[count, j]
            weights.loc[count, j] = 1.0
        else:
            backtest.loc[count, j] = -backtest.loc[count, j]
            weights.loc[count, j] = -1.0
            
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_simp_binary' '_' + 'ls' + '.csv'))


print(summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names))

KNN
Long and Short
                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.009
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                    0.5109
Date:                Sat, 30 Jan 2021   Prob (F-statistic):              0.768
Time:                        11:23:09   Log-Likelihood:                 1674.2
No. Observations:                 613   AIC:                            -3336.
Df Residuals:                     607   BIC:                            -3310.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0272      0.028 

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.116
Model:                            OLS   Adj. R-squared:                  0.109
Method:                 Least Squares   F-statistic:                     7.228
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           1.38e-06
Time:                        11:26:10   Log-Likelihood:                 1702.1
No. Observations:                 613   AIC:                            -3392.
Df Residuals:                     607   BIC:                            -3366.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0482      0.028      1.729      0.0

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     2.489
Date:                Sat, 30 Jan 2021   Prob (F-statistic):             0.0303
Time:                        11:31:40   Log-Likelihood:                 1630.4
No. Observations:                 613   AIC:                            -3249.
Df Residuals:                     607   BIC:                            -3222.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF        -0.0099      0.030     -0.327      0.7

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.132
Model:                            OLS   Adj. R-squared:                  0.125
Method:                 Least Squares   F-statistic:                     7.426
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           8.95e-07
Time:                        11:32:21   Log-Likelihood:                 1749.3
No. Observations:                 613   AIC:                            -3487.
Df Residuals:                     607   BIC:                            -3460.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0778      0.026      2.941      0.0

In [10]:
five_factor_factor_results = summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names)
five_factor_factor_results.title = 'Fama French 5 Factor Model'
five_factor_results_capm = summary_col(output_capm,stars=True,float_format='%0.4f',model_names=names)
five_factor_results_capm.title = 'Famma French capm'
resultFile = open(outname,'a')
resultFile.write(five_factor_factor_results.as_text())
resultFile.write('\n')
resultFile.write(five_factor_results_capm.as_text())
resultFile.write('\n')
resultFile.close()

# Industry

In [11]:
industry = pd.read_csv('48_Industry_Portfolios.csv', index_col = 0)
industry.index = pd.to_datetime(industry.index, format= '%Y%m')
industry[industry <= -99.] = np.nan
industry = industry / 100.0

ff_factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', index_col = 0)
ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m')
ff_factors = ff_factors / 100.0
ff_factors[ff_factors == -999] = np.NaN
ff_factors[ff_factors == -99.9] = np.NaN
ff_factors = ff_factors.dropna(how = 'any', axis = 0)

factor_model = ff_factors.copy().dropna(axis = 0, how = 'any')
rf = factor_model[['RF']]
factor_model = factor_model[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
ff_factors_compare = factor_model.copy()

industry = industry.loc[rf.index,:]
index_symbol = 'Mkt-RF'

print(ff_factors.head(5))

print(industry.head(5))


#start date for data
start = '2004-01-01'
#end date for data
end = '2020-01-01'

#the lookback for alpha feature
lookback = 60
#control the leverage
lev = 1.
stk_symbols  = industry.columns
returns = industry.copy()


#set the backtest parameters
bp = BacktestParameters(stk_symbols, start, end, lookback, lev  )
bt = Backtest(bp, returns, mode = 'insample')
data = Data(bp)

df_alpha, idiosyncratic_vol, binary_alpha = data.get_alpha(returns, factor_model, rf)
rolling_mean_12, rolling_mean_6, rolling_std_12, binary_mean_12,binary_mean_6,binary_returns = data.get_rolling_features(returns, 0.0)


#creates a dictionary of different features for analysis based on ticker keys
feature_list = [binary_alpha]

feature_set = data.get_features(feature_list)


            Mkt-RF     SMB     HML     RMW     CMA      RF
1963-07-01 -0.0039 -0.0047 -0.0083  0.0066 -0.0115  0.0027
1963-08-01  0.0507 -0.0079  0.0167  0.0040 -0.0040  0.0025
1963-09-01 -0.0157 -0.0048  0.0018 -0.0076  0.0024  0.0027
1963-10-01  0.0253 -0.0129 -0.0010  0.0275 -0.0224  0.0029
1963-11-01 -0.0085 -0.0084  0.0171 -0.0045  0.0222  0.0027
             Agric   Food    Soda    Beer    Smoke   Toys    Fun     Books  \
1963-07-01  0.0304 -0.0046  0.0257 -0.0219 -0.0254 -0.0507 -0.0070 -0.0007   
1963-08-01 -0.0032  0.0473  0.0550  0.0214  0.0715  0.0503  0.0784  0.0528   
1963-09-01 -0.0194 -0.0153 -0.0090  0.0123 -0.0423 -0.0515 -0.0226  0.0298   
1963-10-01 -0.0013  0.0269  0.0110 -0.0128  0.0578  0.1566  0.0196 -0.0199   
1963-11-01 -0.0306 -0.0039 -0.0058 -0.0072 -0.0563 -0.0083 -0.0397 -0.0062   

             Hshld   Clths  ...   Boxes   Trans   Whlsl   Rtail   Meals  \
1963-07-01 -0.0015 -0.0067  ...  0.0042 -0.0253  0.0004 -0.0106 -0.0421   
1963-08-01  0.0622  0.0483 

C:\Users\jjanko\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


               Agric     Food      Soda      Beer      Smoke     Toys   \
1968-08-01  0.014818  0.004692  0.015289  0.009995  0.006102  0.008836   
1968-09-01  0.015428  0.002896  0.012857  0.009660  0.003951  0.006772   
1968-10-01  0.015564  0.003582  0.013681  0.010912  0.003847  0.007994   
1968-11-01  0.017528  0.003740  0.013500  0.012015  0.003372  0.005683   
1968-12-01  0.018913  0.004295  0.013894  0.011678  0.004240  0.006426   

               Fun       Books     Hshld     Clths  ...     Boxes     Trans  \
1968-08-01  0.020516  0.018425  0.014672  0.014528  ...  0.009206  0.013008   
1968-09-01  0.018271  0.016216  0.012336  0.013051  ...  0.007369  0.010861   
1968-10-01  0.020349  0.016295  0.012996  0.014904  ...  0.008777  0.011838   
1968-11-01  0.022310  0.017636  0.012503  0.014883  ...  0.008294  0.013259   
1968-12-01  0.023674  0.019380  0.013289  0.016218  ...  0.009948  0.013369   

               Whlsl     Rtail     Meals     Banks     Insur     RlEst  \
1968-0

In [12]:
output_reg = []
output_capm = []
names = ['KNN L/S', 'KNN L', 'KNN S', 'Neural Net L/S', 'Neural Net L', 'Neural Net S', 'RF L/S', 'RF L', 'RF S', 'Ensemble', 'Binary']

data_type = 'industry'

print("KNN")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'ls' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'l' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 's' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Neural Net")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'ls' + '.csv'))

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'l' + '.csv'))

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 's' + '.csv'))

print("Random Forest")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'ls' + '.csv'))

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'l' + '.csv'))

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 's' + '.csv'))

print("Ensemble System")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors", "Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'yes')  
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_ensemble' '_' + 'ls' + '.csv'))
    
print("Simple Binary Decision")
df_alpha_shift = df_alpha.shift(1).dropna(axis = 0)
backtest = returns.loc[df_alpha_shift.index]
weights = returns.loc[df_alpha_shift.index]
for count, row in  backtest.iterrows(): 
    for j in backtest.columns:
        if df_alpha_shift.loc[count, j] >= 0:
            backtest.loc[count, j] = backtest.loc[count, j]
            weights.loc[count, j] = 1.0
        else:
            backtest.loc[count, j] = -backtest.loc[count, j]
            weights.loc[count, j] = -1.0
            
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_simp_binary' '_' + 'ls' + '.csv'))


print(summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names))

KNN
Long and Short
                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.063
Model:                            OLS   Adj. R-squared:                  0.055
Method:                 Least Squares   F-statistic:                     2.836
Date:                Sat, 30 Jan 2021   Prob (F-statistic):             0.0153
Time:                        11:36:24   Log-Likelihood:                 1294.7
No. Observations:                 613   AIC:                            -2577.
Df Residuals:                     607   BIC:                            -2551.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.1800      0.062 

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.352
Model:                            OLS   Adj. R-squared:                  0.347
Method:                 Least Squares   F-statistic:                     12.41
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           1.76e-11
Time:                        12:02:58   Log-Likelihood:                 1288.1
No. Observations:                 613   AIC:                            -2564.
Df Residuals:                     607   BIC:                            -2538.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.5001      0.071      6.997      0.0

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     3.065
Date:                Sat, 30 Jan 2021   Prob (F-statistic):            0.00964
Time:                        12:51:20   Log-Likelihood:                 1279.4
No. Observations:                 613   AIC:                            -2547.
Df Residuals:                     607   BIC:                            -2520.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0562      0.067      0.833      0.4

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.387
Model:                            OLS   Adj. R-squared:                  0.381
Method:                 Least Squares   F-statistic:                     14.74
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           1.18e-13
Time:                        12:58:10   Log-Likelihood:                 1361.1
No. Observations:                 613   AIC:                            -2710.
Df Residuals:                     607   BIC:                            -2684.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.4809      0.060      7.991      0.0

In [13]:
industry_factor_results = summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names)
industry_factor_results.title = 'Industry Returns'
industry_factor_results_capm = summary_col(output_capm,stars=True,float_format='%0.4f',model_names=names)
industry_factor_results_capm.title = 'Industry Returns capm'
resultFile = open(outname,'a')
resultFile.write(industry_factor_results.as_text())
resultFile.write('\n')
resultFile.write(industry_factor_results_capm.as_text())
resultFile.write('\n')
resultFile.close()

# Stambaugh and Yuan "Mispricing Factors"

In [14]:
m4 = pd.read_csv('M4-1.csv', index_col = 0)
m4.index = pd.to_datetime(m4.index, format= '%Y%m')

factor_model = m4.copy()
factor_model = factor_model[['MKTRF','SMB','MGMT','PERF']]
rf = m4[['RF']]

index_symbol = 'MKTRF'

print(m4.head(5))

#start date for data
start = '2004-01-01'
#end date for data
end = '2020-01-01'

#the lookback for alpha feature
lookback = 60
#control the leverage
lev = 1.
stk_symbols  = factor_model.columns
returns = factor_model.copy()


#set the backtest parameters
bp = BacktestParameters(stk_symbols, start, end, lookback, lev  )
bt = Backtest(bp, returns, mode = 'insample')
data = Data(bp)

df_alpha, idiosyncratic_vol, binary_alpha = data.get_alpha(returns, factor_model, rf)
rolling_mean_12, rolling_mean_6, rolling_std_12, binary_mean_12,binary_mean_6,binary_returns = data.get_rolling_features(returns, 0.0)


#creates a dictionary of different features for analysis based on ticker keys
feature_list = [binary_alpha]

feature_set = data.get_features(feature_list)


             MKTRF       SMB      MGMT      PERF      RF
YYYYMM                                                  
1963-01-01  0.0493  0.024818  0.021349 -0.022751  0.0025
1963-02-01 -0.0238  0.018096  0.010961  0.000874  0.0023
1963-03-01  0.0308 -0.008183  0.005890  0.023281  0.0023
1963-04-01  0.0451 -0.037328  0.000190  0.005077  0.0025
1963-05-01  0.0176  0.021518  0.017593  0.002848  0.0024


C:\Users\jjanko\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


               MKTRF       SMB      MGMT      PERF
YYYYMM                                            
1968-02-01  0.005765  0.007119 -0.005921  0.005407
1968-03-01  0.005814  0.007359 -0.005636  0.005246
1968-04-01  0.004788  0.007196 -0.004694  0.004625
1968-05-01  0.004102  0.007359 -0.005246  0.005616
1968-06-01  0.005354  0.007753 -0.007145  0.007251
               MKTRF       SMB      MGMT      PERF
YYYYMM                                            
1968-02-01  0.027592  0.026434  0.021556  0.024572
1968-03-01  0.028094  0.026578  0.023250  0.025837
1968-04-01  0.028262  0.026593  0.023866  0.025988
1968-05-01  0.027668  0.025969  0.023951  0.026408
1968-06-01  0.029633  0.026750  0.024786  0.027249


In [15]:
output_reg = []
output_capm = []
names = ['KNN L/S', 'KNN L', 'KNN S', 'Neural Net L/S', 'Neural Net L', 'Neural Net S', 'RF L/S', 'RF L', 'RF S', 'Ensemble', 'Binary']

data_type = 'stambaugh_yuan'

print("KNN")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'ls' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'l' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 's' + '.csv'))
#backtest.mean(axis = 1).cumsum().plot()

print("Neural Net")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'ls' + '.csv'))

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'l' + '.csv'))

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 's' + '.csv'))

print("Random Forest")
print("Long and Short")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'ls' + '.csv'))

print("Long Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'l' + '.csv'))

print("Short Only")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 's' + '.csv'))

print("Ensemble System")
backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors", "Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'yes')  
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_ensemble' '_' + 'ls' + '.csv'))
    
print("Simple Binary Decision")
df_alpha_shift = df_alpha.shift(1).dropna(axis = 0)
backtest = returns.loc[df_alpha_shift.index]
weights = returns.loc[df_alpha_shift.index]
for count, row in  backtest.iterrows(): 
    for j in backtest.columns:
        if df_alpha_shift.loc[count, j] >= 0:
            backtest.loc[count, j] = backtest.loc[count, j]
            weights.loc[count, j] = 1.0
        else:
            backtest.loc[count, j] = -backtest.loc[count, j]
            weights.loc[count, j] = -1.0
            
results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
output_reg.append(results)
output_capm.append(results_capm)
#backtest.mean(axis = 1).cumsum().plot()
backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_simp_binary' '_' + 'ls' + '.csv'))


print(summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names))

KNN
Long and Short
                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.032
Model:                            OLS   Adj. R-squared:                  0.023
Method:                 Least Squares   F-statistic:                     1.966
Date:                Sat, 30 Jan 2021   Prob (F-statistic):             0.0819
Time:                        12:58:40   Log-Likelihood:                 1466.2
No. Observations:                 575   AIC:                            -2920.
Df Residuals:                     569   BIC:                            -2894.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF        -0.0121      0.034 

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.090
Method:                 Least Squares   F-statistic:                     6.038
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           1.85e-05
Time:                        13:00:49   Log-Likelihood:                 1475.5
No. Observations:                 575   AIC:                            -2939.
Df Residuals:                     569   BIC:                            -2913.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0206      0.037      0.551      0.5

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.101
Model:                            OLS   Adj. R-squared:                  0.093
Method:                 Least Squares   F-statistic:                     4.500
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           0.000496
Time:                        13:04:45   Log-Likelihood:                 1431.5
No. Observations:                 575   AIC:                            -2851.
Df Residuals:                     569   BIC:                            -2825.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF        -0.0244      0.037     -0.661      0.5

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.096
Model:                            OLS   Adj. R-squared:                  0.088
Method:                 Least Squares   F-statistic:                     4.721
Date:                Sat, 30 Jan 2021   Prob (F-statistic):           0.000310
Time:                        13:05:17   Log-Likelihood:                 1500.1
No. Observations:                 575   AIC:                            -2988.
Df Residuals:                     569   BIC:                            -2962.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.0305      0.035      0.869      0.3

In [16]:
m4_factor_results = summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names)
m4_factor_results.title = 'RF Stambaugh, Y Yuan 4 Factor Model'
m4_factor_results_capm = summary_col(output_capm,stars=True,float_format='%0.4f',model_names=names)
m4_factor_results_capm.title = 'RF Stambaugh, Y Yuan 4 Factor Model capm'
resultFile = open(outname,'a')
resultFile.write(m4_factor_results.as_text())
resultFile.write('\n')
resultFile.write(m4_factor_results_capm.as_text())
resultFile.write('\n')
resultFile.close()

# Testing Portfolios- Hou, Kewei, Chen Xue, and Lu Zhang

http://global-q.org/testingportfolios.html

In [17]:
data_dir = os.path.join(os.getcwd(), 'zhang_anomalies')
_, _, filenames = next(os.walk(data_dir))
print("Testing Portfolio files")
print('Show the first 5...')
print(filenames[0:5])

for three_sort in range(1,4):

    for five_sort in range(1,6):

        anomalies = pd.DataFrame()

        for fil in filenames:
            file_path = os.path.join(data_dir, fil)
            df = pd.read_csv(file_path)
            df["month"] = df["month"].astype(str)
            for i in range(9):
                df.loc[df["month"] == str(i), 'month'] = "0" + df.loc[df["month"] == str(i), 'month']
            df.loc[:, 'ret_vw'] = df.loc[:, 'ret_vw'] / 100.0
            df['date'] = 0.0
            df['date'] = df["year"].astype(str) + df["month"].astype(str) 
            df.index = pd.to_datetime(df.date, format= '%Y%m')

            three_sort_name = ''
            five_sort_name = ''
            col_names = df.columns
            for i in col_names:
                if 'rank' in i and 5 in df[i].unique():
                    five_sort_name = i
                if 'rank' in i and 5 not in df[i].unique():
                    three_sort_name = i
            df_add = df.loc[df[three_sort_name] == three_sort]
            df_add = df_add.loc[df_add[five_sort_name] == five_sort]
            df_add = pd.DataFrame(df_add['ret_vw'])
            df_add.columns = [fil.replace('_monthly_2019a.csv', '')]
            if anomalies.empty:
                anomalies = df_add.copy()
            else:
                anomalies = pd.concat([anomalies, df_add], axis = 1)
                
        ff_factors = pd.read_csv('F-F_Research_Data_5_Factors_2x3.csv', index_col = 0)
        ff_factors.index = pd.to_datetime(ff_factors.index, format= '%Y%m')
        ff_factors = ff_factors / 100.0
        ff_factors[ff_factors == -999] = np.NaN
        ff_factors[ff_factors == -99.9] = np.NaN
        ff_factors = ff_factors.dropna(how = 'any', axis = 0)
        ff_factors = ff_factors.dropna(axis = 0, how = 'any')
        ff_factors = ff_factors.loc[anomalies.index].dropna(axis = 0)
        
        factor_model = ff_factors.copy().dropna(axis = 0, how = 'any')
        rf = factor_model[['RF']]
        factor_model = factor_model[['Mkt-RF', 'SMB', 'HML', 'RMW', 'CMA']]
        print(factor_model)
        print(rf)

        ff_factors_compare = factor_model.copy()
        
        anomalies = anomalies.loc[:, anomalies.columns[0:30]]
        #start date for data
        start = '2004-01-01'
        #end date for data
        end = '2020-01-01'

        #the lookback for alpha feature
        lookback = 60
        #control the leverage
        lev = 1.
        stk_symbols  = anomalies.columns
        returns = anomalies.copy()


        #set the backtest parameters
        bp = BacktestParameters(stk_symbols, start, end, lookback, lev  )
        bt = Backtest(bp, returns, mode = 'insample')
        data = Data(bp)

        df_alpha, idiosyncratic_vol, binary_alpha = data.get_alpha(returns, factor_model, rf)
        rolling_mean_12, rolling_mean_6, rolling_std_12, binary_mean_12,binary_mean_6,binary_returns = data.get_rolling_features(returns, 0.0)


        #creates a dictionary of different features for analysis based on ticker keys
        feature_list = [binary_alpha]
        #feature_list = [df_alpha]

        feature_set = data.get_features(feature_list)


        output_reg = []
        output_capm = []
        names = ['KNN L/S', 'KNN L', 'KNN S', 'Neural Net L/S', 'Neural Net L', 'Neural Net S', 'RF L/S', 'RF L', 'RF S', 'Ensemble', 'Binary']

        data_type = 'anomaly_port' + '_' + str(three_sort) + '_' + str(five_sort)

        print("KNN")
        print("Long and Short")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'ls' + '.csv'))
        #backtest.mean(axis = 1).cumsum().plot()

        print("Long Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 'l' + '.csv'))
        #backtest.mean(axis = 1).cumsum().plot()

        print("Short Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_KNN' '_' + 's' + '.csv'))
        #backtest.mean(axis = 1).cumsum().plot()

        print("Neural Net")
        print("Long and Short")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'ls' + '.csv'))

        print("Long Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 'l' + '.csv'))

        print("Short Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Neural Net"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_NN' '_' + 's' + '.csv'))

        print("Random Forest")
        print("Long and Short")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'both')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'ls' + '.csv'))

        print("Long Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'long')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 'l' + '.csv'))

        print("Short Only")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'no', trade_type = 'short')
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_RF' '_' + 's' + '.csv'))

        print("Ensemble System")
        backtest, weights = bt.run_backtest(returns, input_classifiers= ["Nearest Neighbors", "Random Forest"], lookback_backtest = 12, inputs = feature_set, outputs = binary_returns,  ensemble = 'yes')  
        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_ensemble' '_' + 'ls' + '.csv'))

        print("Simple Binary Decision")
        df_alpha_shift = df_alpha.shift(1).dropna(axis = 0)
        backtest = returns.loc[df_alpha_shift.index]
        weights = returns.loc[df_alpha_shift.index]
        for count, row in  backtest.iterrows(): 
            for j in backtest.columns:
                if df_alpha_shift.loc[count, j] >= 0:
                    backtest.loc[count, j] = backtest.loc[count, j]
                    weights.loc[count, j] = 1.0
                else:
                    backtest.loc[count, j] = -backtest.loc[count, j]
                    weights.loc[count, j] = -1.0

        results, fm_sum = bt.factor_results(backtest, ff_factors_compare)
        results_capm, fm_sum_capm = bt.factor_results_capm(backtest, ff_factors_compare)
        output_reg.append(results)
        output_capm.append(results_capm)
        #backtest.mean(axis = 1).cumsum().plot()
        backtest.to_csv(os.path.join(cur_wd, 'backtests', data_type + '_simp_binary' '_' + 'ls' + '.csv'))


        print(summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names))

        tp_factor_results = summary_col(output_reg,stars=True,float_format='%0.4f',model_names=names)
        tp_factor_results.title = 'Testing Portfolios- Hou, Kewei, Chen Xue, and Lu Zhang' + ' ' + str(three_sort) + ' ' + str(five_sort)
        tp_factor_results_capm = summary_col(output_capm,stars=True,float_format='%0.4f',model_names=names)
        tp_factor_results_capm.title = 'Testing Portfolios- Hou, Kewei, Chen Xue, and Lu Zhang' + ' ' + str(three_sort) + ' ' + str(five_sort) + ' ' + 'capm'
        resultFile = open(outname,'a')
        resultFile.write(tp_factor_results.as_text())
        resultFile.write('\n')
        resultFile.write(tp_factor_results_capm.as_text())
        resultFile.write('\n')
        resultFile.close()




Testing Portfolio files
Show the first 5...
['mom_portf_me_abr_1_monthly_2019a(1).csv', 'portf_me_abr_12_monthly_2019a(1).csv', 'portf_me_abr_12_monthly_2019a.csv', 'portf_me_abr_1_monthly_2019a(1).csv', 'portf_me_abr_1_monthly_2019a.csv']
            Mkt-RF     SMB     HML     RMW     CMA
date                                              
1967-01-01  0.0815  0.0900  0.0222  0.0062 -0.0297
1967-02-01  0.0078  0.0303 -0.0217  0.0194 -0.0094
1967-03-01  0.0399  0.0190  0.0031  0.0090 -0.0151
1967-04-01  0.0389  0.0046 -0.0264  0.0243 -0.0375
1967-05-01 -0.0433  0.0237  0.0080 -0.0175  0.0161
1967-06-01  0.0241  0.0646  0.0096 -0.0064 -0.0239
1967-07-01  0.0458  0.0348  0.0265  0.0051  0.0272
1967-08-01 -0.0089  0.0076  0.0146  0.0042  0.0141
1967-09-01  0.0311  0.0246 -0.0247  0.0022 -0.0095
1967-10-01 -0.0309  0.0047 -0.0339  0.0081 -0.0255
1967-11-01  0.0037 -0.0006 -0.0171  0.0129 -0.0229
1967-12-01  0.0305  0.0575 -0.0039 -0.0064  0.0003
1968-01-01 -0.0406  0.0448  0.0475 -0.0458  0.

C:\Users\jjanko\AppData\Local\Continuum\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:2389: FutureWarning: Method .ptp is deprecated and will be removed in a future version. Use numpy.ptp instead.
  return ptp(axis=axis, out=out, **kwargs)


            mom_portf_me_abr_1_monthly_2019a(1).csv  \
date                                                  
1972-02-01                                      NaN   
1972-03-01                                 0.116478   
1972-04-01                                 0.057935   
1972-05-01                                -0.054002   
1972-06-01                                 0.026850   

            portf_me_abr_12_monthly_2019a(1).csv  portf_me_abr_12  \
date                                                                
1972-02-01                                   NaN              NaN   
1972-03-01                              0.116478         0.116478   
1972-04-01                              0.056445         0.056445   
1972-05-01                             -0.051114        -0.051114   
1972-06-01                              0.028260         0.028260   

            portf_me_abr_1_monthly_2019a(1).csv  portf_me_abr_1  \
date                                                           

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.900
Model:                            OLS   Adj. R-squared:                  0.899
Method:                 Least Squares   F-statistic:                     710.2
Date:                Sat, 30 Jan 2021   Prob (F-statistic):          7.26e-228
Time:                        13:08:49   Log-Likelihood:                 1250.5
No. Observations:                 519   AIC:                            -2489.
Df Residuals:                     513   BIC:                            -2463.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.9887      0.036     27.200      0.0

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.927
Model:                            OLS   Adj. R-squared:                  0.927
Method:                 Least Squares   F-statistic:                     817.7
Date:                Sat, 30 Jan 2021   Prob (F-statistic):          7.86e-248
Time:                        13:38:51   Log-Likelihood:                 1419.4
No. Observations:                 541   AIC:                            -2827.
Df Residuals:                     535   BIC:                            -2801.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         1.0013      0.025     40.826      0.0

...

            mom_portf_me_abr_1_monthly_2019a(1).csv  \
date                                                  
1972-02-01                                      NaN   
1972-03-01                                 0.040132   
1972-04-01                                 0.044592   
1972-05-01                                 0.008143   
1972-06-01                                 0.035297   

            portf_me_abr_12_monthly_2019a(1).csv  portf_me_abr_12  \
date                                                                
1972-02-01                                   NaN              NaN   
1972-03-01                              0.040132         0.040132   
1972-04-01                              0.039869         0.039869   
1972-05-01                              0.003532         0.003532   
1972-06-01                              0.030673         0.030673   

            portf_me_abr_1_monthly_2019a(1).csv  portf_me_abr_1  \
date                                                           

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.919
Model:                            OLS   Adj. R-squared:                  0.918
Method:                 Least Squares   F-statistic:                     822.8
Date:                Sun, 31 Jan 2021   Prob (F-statistic):          1.56e-246
Time:                        00:31:54   Log-Likelihood:                 1545.5
No. Observations:                 534   AIC:                            -3079.
Df Residuals:                     528   BIC:                            -3053.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.9930      0.017     59.658      0.0

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.942
Model:                            OLS   Adj. R-squared:                  0.942
Method:                 Least Squares   F-statistic:                     1439.
Date:                Sun, 31 Jan 2021   Prob (F-statistic):          9.15e-313
Time:                        00:59:26   Log-Likelihood:                 1712.6
No. Observations:                 555   AIC:                            -3413.
Df Residuals:                     549   BIC:                            -3387.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.9951      0.013     75.334      0.0

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.891
Model:                            OLS   Adj. R-squared:                  0.890
Method:                 Least Squares   F-statistic:                     478.0
Date:                Sun, 31 Jan 2021   Prob (F-statistic):          1.42e-192
Time:                        01:14:54   Log-Likelihood:                 1447.9
No. Observations:                 530   AIC:                            -2884.
Df Residuals:                     524   BIC:                            -2858.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         1.0224      0.025     40.492      0.0

                            OLS Regression Results                            
Dep. Variable:              portfolio   R-squared:                       0.865
Model:                            OLS   Adj. R-squared:                  0.863
Method:                 Least Squares   F-statistic:                     289.0
Date:                Sun, 31 Jan 2021   Prob (F-statistic):          8.32e-142
Time:                        01:17:34   Log-Likelihood:                 1306.5
No. Observations:                 483   AIC:                            -2601.
Df Residuals:                     477   BIC:                            -2576.
Df Model:                           5                                         
Covariance Type:                  HAC                                         
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
Mkt-RF         0.8858      0.029     30.869      0.0